In [1]:
import os
import re
import networkx as nx
import pickle
from tqdm import tqdm

def list_iteration_snapshots(output_dir, save_prefix, filetype='graphml'):
    # List and sort files matching the pattern
    pattern = re.compile(rf"{re.escape(save_prefix)}_iter(\d+)\.{filetype}$")
    all_files = [
        fname for fname in os.listdir(output_dir)
        if pattern.match(fname)
    ]
    # Sort by iteration number
    all_files_sorted = sorted(
        all_files,
        key=lambda f: int(pattern.match(f).group(1))
    )
    return [os.path.join(output_dir, fname) for fname in all_files_sorted]

def load_graph_snapshot(path):
    ext = os.path.splitext(path)[1]
    if ext == '.graphml':
        return nx.read_graphml(path)
    elif ext in ('.pkl', '.pickle'):
        with open(path, 'rb') as f:
            return pickle.load(f)
    else:
        raise ValueError("Unknown graph file format: " + ext)

# --- EXAMPLE USAGE ---
output_dir = os.path.join('..', 'Data', 'Output')
save_prefix = 'nx_semantic'  # Adjust to your actual prefix!
filetype = 'graphml'         # Or 'pkl' as needed

snapshot_files = list_iteration_snapshots(output_dir, save_prefix, filetype)
print(f"Found {len(snapshot_files)} graph snapshots.")

# Load each snapshot, analyze or visualize as needed
for i, snap_path in enumerate(tqdm(snapshot_files, desc="Processing snapshots")):
    G = load_graph_snapshot(snap_path)
    # ...do your visualization/export/animation here!
    print(f"Iteration {i+1}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")


Found 3 graph snapshots.


Processing snapshots:  33%|███▎      | 1/3 [00:02<00:04,  2.23s/it]

Iteration 1: 9992 nodes, 155079 edges


Processing snapshots:  67%|██████▋   | 2/3 [00:05<00:03,  3.01s/it]

Iteration 2: 19898 nodes, 222802 edges


Processing snapshots: 100%|██████████| 3/3 [00:11<00:00,  3.68s/it]

Iteration 3: 29769 nodes, 279422 edges


In [2]:
import os
import json
import networkx as nx
import numpy as np
from tqdm import tqdm

def compute_3d_layout(G, seed=42):
    """Compute a 3D force-directed layout for the graph, returns dict[node] = (x,y,z)."""
    pos_2d = nx.spring_layout(G, dim=3, seed=seed)  # 3D spring layout
    return pos_2d

def graph_to_threejs_json(G, pos, frame_idx=0, cluster_labels=None, max_neighbors=10):
    """Convert a graph and positions to a Three.js-ready JSON structure."""
    nodes = []
    node_id_map = {n: idx for idx, n in enumerate(G.nodes())}
    for n, data in G.nodes(data=True):
        degree = G.degree(n)
        cluster = int(cluster_labels[n]) if cluster_labels else 0
        nodes.append({
            "id": node_id_map[n],
            "name": str(n),
            "x": float(pos[n][0]),
            "y": float(pos[n][1]),
            "z": float(pos[n][2]),
            "degree": degree,
            "cluster": cluster
        })
    # Only include up to max_neighbors edges per node for performance
    edges = []
    for n1, n2, data in G.edges(data=True):
        if node_id_map[n1] < node_id_map[n2]:  # Avoid double-edges
            edges.append({
                "source": node_id_map[n1],
                "target": node_id_map[n2]
            })
    return {
        "frame": frame_idx,
        "nodes": nodes,
        "edges": edges
    }

# --- PARAMETERS ---
output_dir = os.path.join('..', 'Data', 'Output')
save_prefix = 'nx_semantic'  # Adjust as needed
filetype = 'graphml'
json_out_dir = './threejs_snapshots'
os.makedirs(json_out_dir, exist_ok=True)

snapshot_files = list_iteration_snapshots(output_dir, save_prefix, filetype)
print(f"Found {len(snapshot_files)} graph snapshots.")

# OPTIONAL: For cluster coloring, you could use networkx community detection per frame.
def get_communities(G):
    # Fast, simple Louvain community detection (install python-louvain if needed)
    try:
        import community as community_louvain
        part = community_louvain.best_partition(G.to_undirected())
    except ImportError:
        part = {n: 0 for n in G.nodes()}
    return part

for i, snap_path in enumerate(tqdm(snapshot_files, desc="Exporting JSON frames for Three.js")):
    G = load_graph_snapshot(snap_path)
    pos = compute_3d_layout(G)
    cluster_labels = get_communities(G)
    jsdata = graph_to_threejs_json(G, pos, frame_idx=i, cluster_labels=cluster_labels)
    # Write to file for browser use
    json_path = os.path.join(json_out_dir, f"frame_{i:02d}.json")
    with open(json_path, "w") as f:
        json.dump(jsdata, f)
    print(f"Frame {i}: {len(G.nodes())} nodes, {len(G.edges())} edges → {json_path}")


Found 3 graph snapshots.


Exporting JSON frames for Three.js:   0%|          | 0/3 [01:32<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Semantic Graph Evolution</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <style>
        body { margin: 0; background: #111; overflow: hidden; color: #fff; }
        #controls { position: fixed; top: 20px; left: 20px; z-index: 1000; }
        #frameLabel { font-size: 1.4em; margin-left: 8px; }
    </style>
</head>
<body>
    <div id="controls">
        <button onclick="prevFrame()">⏮️ Prev</button>
        <button onclick="nextFrame()">Next ⏭️</button>
        <span id="frameLabel">Frame: 0</span>
    </div>
    <canvas id="canvas"></canvas>
    <script>
        let frames = [];
        let currentFrame = 0;
        let scene, camera, renderer, nodeObjects = [], edgeObjects = [];
        const maxFrames = 20;  // update after loading files

        // Load all JSON frames
        async function loadAllFrames() {
            let loaded = 0;
            while (true) {
                let fname = `threejs_snapshots/frame_${String(loaded).padStart(2,'0')}.json`;
                try {
                    let resp = await fetch(fname);
                    if (!resp.ok) break;
                    let data = await resp.json();
                    frames.push(data);
                    loaded++;
                } catch (e) { break; }
            }
            document.getElementById('frameLabel').textContent = `Frame: 0/${frames.length-1}`;
            if (frames.length > 0) showFrame(0);
        }

        // Initialize Three.js scene
        function initThree() {
            scene = new THREE.Scene();
            camera = new THREE.PerspectiveCamera(75, window.innerWidth/window.innerHeight, 0.1, 2000);
            camera.position.set(0, 0, 400);
            renderer = new THREE.WebGLRenderer({canvas: document.getElementById('canvas'), antialias: true});
            renderer.setSize(window.innerWidth, window.innerHeight);
            renderer.setClearColor(0x111111);
            animate();
        }
        function animate() {
            requestAnimationFrame(animate);
            renderer.render(scene, camera);
        }

        // Draw the current frame's nodes/edges
        function showFrame(idx) {
            // Clear previous objects
            nodeObjects.forEach(o => scene.remove(o));
            edgeObjects.forEach(o => scene.remove(o));
            nodeObjects = [];
            edgeObjects = [];
            if (idx < 0 || idx >= frames.length) return;
            const f = frames[idx];
            // Draw edges
            f.edges.forEach(e => {
                let src = f.nodes[e.source], tgt = f.nodes[e.target];
                let material = new THREE.LineBasicMaterial({ color: 0x888888 });
                let geom = new THREE.BufferGeometry().setFromPoints([
                    new THREE.Vector3(src.x, src.y, src.z),
                    new THREE.Vector3(tgt.x, tgt.y, tgt.z)
                ]);
                let line = new THREE.Line(geom, material);
                edgeObjects.push(line); scene.add(line);
            });
            // Draw nodes (colored by cluster)
            const palette = [0xffd700, 0x1e90ff, 0xa020f0, 0x32cd32, 0xff69b4, 0xff4500, 0x00fa9a, 0xb22222, 0x4682b4, 0x20b2aa];
            f.nodes.forEach(n => {
                let color = palette[n.cluster % palette.length];
                let mat = new THREE.MeshBasicMaterial({ color });
                let geom = new THREE.SphereGeometry(2 + Math.log2(n.degree+1), 16, 16);
                let mesh = new THREE.Mesh(geom, mat);
                mesh.position.set(n.x, n.y, n.z);
                nodeObjects.push(mesh); scene.add(mesh);
            });
            currentFrame = idx;
            document.getElementById('frameLabel').textContent = `Frame: ${idx}/${frames.length-1}`;
        }
        function prevFrame() { if (currentFrame > 0) showFrame(currentFrame - 1); }
        function nextFrame() { if (currentFrame < frames.length - 1) showFrame(currentFrame + 1); }

        window.addEventListener('resize', () => {
            camera.aspect = window.innerWidth / window.innerHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(window.innerWidth, window.innerHeight);
        });

        // Initialize!
        initThree();
        loadAllFrames();
    </script>
</body>
</html>